In [ ]:
# 魔术指令，自动加载模块
%load_ext autoreload
%autoreload 2
import os
os.chdir("/home/beihang/xihu/HZTourism/FlowPred-dev")
from utils.utils_data import save_csv_from_db
import pandas as pd
from utils.utils_data import fill_missing_value_singlespot_day
from pattern.pattern_train import get_group_annotation, save_mode_data

## 原始数据获取

In [ ]:
table_name_lst= ["mobile_signaling_tourists_num","dahua_flow","lingyin_passenger_flow"]

for table_name in table_name_lst:
    if table_name=="mobile_signaling_tourists_num":
        time_col = "kpi_time"
    elif table_name=="dahua_flow":
        time_col = "date_time"
    save_csv_from_db(
        table_name=table_name,
        time_col=time_col,
        s_time="2024-07-19 00:00:00",
        e_time="2025-07-20 00:00:00",
        output_csv_file=f"data/pred6h_launch/raw/{table_name}_240719_250720.csv"
    )


## 数据预处理

In [ ]:
# 信令客流处理
df_mobile = pd.read_csv("data/pred6h_launch/raw/mobile_signaling_tourists_num_240719_250720.csv")
df_mobile["kpi_time"] = pd.to_datetime(df_mobile["kpi_time"])
df_mobile.drop_duplicates(subset=["spot_id","kpi_time"], inplace=True)
spot_id_lst = [14100,14102,14103,14105,14107,14108,14114,14115,14116,14120,14124,14125,14126,14127,14129,14137,14141,14144,14145,14205]

for spot_id in spot_id_lst:
    df_mobile_spot = df_mobile[df_mobile["spot_id"]==spot_id]
    df_mobile_spot = df_mobile_spot[["spot_id","kpi_time","kpi_value"]]
    df_mobile_spot = fill_missing_value_singlespot_day(df_mobile_spot)
    df_mobile_spot.to_csv(f"data/pred6h_launch/raw/mobile_spot/{spot_id}_240719_250720.csv", index=False)

In [ ]:
# 填充函数
def fill_missing_value_singlespot_30s(spot_id,df):
    df['minute_str'] = df["kpi_time"].dt.strftime('%Y-%m-%d %H:%M')
    df_groups = list(df.groupby('minute_str'))
    kpi_time_lst = []
    kpi_value_lst = []
    for minute_str, group in df_groups:
        if len(group) == 1:
            # 依据minute_str生成两个时间戳，其他都一样，秒数一个是00，一个是30
            kpi_time_lst.append(group['kpi_time'].iloc[0].replace(second=0, microsecond=0))
            kpi_time_lst.append(group['kpi_time'].iloc[0].replace(second=30, microsecond=0))
            kpi_value_lst.append(group['kpi_value'].iloc[0])
            kpi_value_lst.append(group['kpi_value'].iloc[0])
        elif len(group) == 2:
            kpi_time_lst.append(group['kpi_time'].iloc[0].replace(second=0, microsecond=0))
            kpi_time_lst.append(group['kpi_time'].iloc[0].replace(second=30, microsecond=0))
            kpi_value_lst.append(group['kpi_value'].iloc[0])
            kpi_value_lst.append(group['kpi_value'].iloc[1])
        elif len(group) > 2:
            df_s0 = group[group['kpi_time'].dt.second == 0]
            df_s30 = group[group['kpi_time'].dt.second == 30]
            # 初始化为 
            kpi_value_0 = None
            kpi_value_30 = None
            if len(df_s0) > 0:
                kpi_value_0 = df_s0['kpi_value'].iloc[0]
            if len(df_s30) > 0:
                kpi_value_30 = df_s30['kpi_value'].iloc[0]
            if kpi_value_0 is None:
                kpi_value_0 = group['kpi_value'].iloc[0]
            if kpi_value_30 is None:
                kpi_value_30 = group['kpi_value'].iloc[-1]
            kpi_time_lst.append(group['kpi_time'].iloc[0].replace(second=0, microsecond=0))
            kpi_time_lst.append(group['kpi_time'].iloc[0].replace(second=30, microsecond=0))
            kpi_value_lst.append(kpi_value_0)
            kpi_value_lst.append(kpi_value_30)
    df_res = pd.DataFrame({
        'spot_id': [spot_id] * len(kpi_time_lst),
        'kpi_time': kpi_time_lst,
        'kpi_value': kpi_value_lst
    })
    df_res['kpi_time'] = pd.to_datetime(df_res['kpi_time'])
    return df_res

In [ ]:
# 断桥数据初始处理
df_dh = pd.read_csv("data/pred6h_launch/raw/dahua_flow_240719_250720.csv")
df_dh = df_dh[["date_time","num"]]
df_dh.rename(columns={"date_time": "kpi_time", "num": "kpi_value"}, inplace=True)
df_dh["spot_id"] = 14207
df_dh["kpi_time"] = pd.to_datetime(df_dh["kpi_time"])
df_dh.drop_duplicates(subset=["kpi_time"], inplace=True)
df_dh['minute_str'] = df_dh["kpi_time"].dt.strftime('%Y-%m-%d %H:%M')
df_dh = df_dh.sort_values(['minute_str', 'kpi_value']).drop_duplicates(['minute_str', 'kpi_value'])
df_dh_filled = fill_missing_value_singlespot_30s(14207, df_dh)
df_dh_filled.to_csv("data/pred6h_launch/raw/proc/14207_240719_250720.csv", index=False)

In [ ]:
# 灵隐寺数据初始处理
df_ly = pd.read_csv("data/pred6h_launch/raw/lingyin_passenger_flow_240719_250720.csv")
df_ly = df_ly[["date_time","real_time_num"]]
df_ly.rename(columns={"date_time": "kpi_time", "real_time_num": "kpi_value"}, inplace=True)
df_ly["spot_id"] = 14208
df_ly["kpi_time"] = pd.to_datetime(df_ly["kpi_time"])
df_ly.drop_duplicates(subset=["kpi_time"], inplace=True)
df_ly['minute_str'] = df_ly["kpi_time"].dt.strftime('%Y-%m-%d %H:%M')
df_ly = df_ly.sort_values(['minute_str', 'kpi_value']).drop_duplicates(['minute_str', 'kpi_value'])
df_ly_filled = fill_missing_value_singlespot_30s(14208, df_ly)
df_ly_filled.to_csv("data/pred6h_launch/raw/proc/14208_240719_250720.csv", index=False)

In [26]:
# 按分钟分组，之后统计每分钟的数据个数，生成一个字典保存在temp文件夹下
def group_by_minute_str(df):
    time_col = "kpi_time"
    df[time_col] = pd.to_datetime(df[time_col])
    # 以“年月日时分”字符串为分组依据
    df['minute_str'] = df[time_col].dt.strftime('%Y-%m-%d %H:%M')
    count_per_minute = df.groupby('minute_str').size()
    # 保存在temp文件夹下
    count_per_minute.to_csv("temp/count_per_minute.csv", header=True)
    count_distribution = count_per_minute.value_counts().sort_index().to_dict()
    return count_per_minute, count_distribution

## 模型输入处理

In [ ]:
df_dir = "data/pred6h_lanuch/raw/proc"
# df_files = sorted(os.listdir(df_dir))
df_files = ["14207_240719_250720.csv", "14208_240719_250720.csv"]

for df_file in df_files:
    spot_id = int(df_file.split('_')[0]) 
    his_len = 288
    pred_len = 72
    time_interval="5min"
    if spot_id in [14207, 14208]:
        his_len = 2880
        pred_len = 720
        time_interval="30s"
    data_save_basedir = f"data/pred6h_lanuch/input/{str(spot_id)}"
    df = pd.read_csv(os.path.join(df_dir, df_file))
    groups_mode_0, groups_mode_1 = get_group_annotation(his_len=his_len,pred_len=pred_len, df=df, time_interval=time_interval)
    save_mode_data(
        groups_mode=groups_mode_0,
        mode=0,
        his_len=his_len,
        pred_len=pred_len,
        data_basepath=data_save_basedir,
    )
    save_mode_data(
        groups_mode=groups_mode_1,
        mode=1,
        his_len=his_len,
        pred_len=pred_len,
        data_basepath=data_save_basedir,
    )

: 